In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.3 MB/s eta 0:00:00


In [ ]:
import sentencepiece as spm

# Define input and output file paths
input_file = '/content/drive/MyDrive/StoryTellingMiniGPT/englishWords-Required.txt'
output_model_prefix = '/content/drive/MyDrive/StoryTellingMiniGPT/tokenizerWild'

# Train SentencePiece model
spm.SentencePieceTrainer.Train(f'--input={input_file} --model_prefix={output_model_prefix} --user_defined_symbols=["<q>","</q>","<a>","</a>","<|endoftext|>"] --vocab_size=8000')

# Load trained model
sp = spm.SentencePieceProcessor()
sp.Load(f'{output_model_prefix}.model')



True

In [ ]:
# Tokenize a sentence
# encode: text => id
print(sp.encode('<a>This is a test</a><|endoftext|> and <q>I am me</q>'))
print(sp.encode_as_pieces('<a>This is a test</a><|endoftext|> and <q>I am me</q>'))


[8, 5, 2847, 28, 15, 596, 6, 7, 13, 150, 84, 50, 91, 361, 113, 4]
['▁', '<a>', 'This', '▁is', '▁a', '▁test', '</a>', '<|endoftext|>', '▁and', '▁<', 'q', '>', 'I', '▁am', '▁me', '</q>']


In [ ]:
# !pip install sentencepiece
import sentencepiece as spm
import torch
import torch.nn as nn
from torch.nn import functional as F
file = open('/content/drive/MyDrive/StoryTellingMiniGPT/englishWords-Required.txt', 'r', encoding ='utf-8')
text = file.read()
sp = spm.SentencePieceProcessor(model_file='/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/wild_tokeniser.model')

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 32
n_layer = 8
dropout = 0.1
# ------------

torch.manual_seed(1337)

chars = sorted(list(set(text)))
vocab_size = 8000
def encode(input):
  return sp.encode(input)
def decode(output):
  return sp.decode(output)

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

def save_checkpoint(epoch, model, optimizer, filename='/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/model_checkpoint.pth'):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

model = BigramLanguageModel()

# Load only the 'model_state_dict' from the checkpoint file
checkpoint_path = '/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/model_weights_epoch_300.pth'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))  # Specify 'map_location' based on your setup

# Load the 'model_state_dict' into your model
model.load_state_dict(checkpoint['model_state_dict'])

m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    train_loss=[]
    val_loss = []
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        train_loss.append(losses['train'])
        val_loss.append(losses['val'])
        # save_checkpoint(iter, model, optimizer, filename=f'/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/model_weights_epoch_{iter}.pth')
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    save_checkpoint(iter, model, optimizer, filename=f'/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/model_weights_epoch_.pth')

10.481984 M parameters
step 0: train loss 5.9352, val loss 5.6776
step 300: train loss 5.3638, val loss 5.2313


KeyboardInterrupt: ignored

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=100)[0].tolist()))

NameError: ignored

# Inference

In [ ]:
import sentencepiece as spm
import torch
import torch.nn as nn
from torch.nn import functional as F
# file = open('/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/512-full-prompt-answer.txt', 'r', encoding ='utf-8')
# text = file.read()
sp = spm.SentencePieceProcessor(model_file='/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/wild_tokeniser_ver2.model')


In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 512 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 300
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 32
n_layer = 8
dropout = 0.1
vocab_size = 20000
# ------------

torch.manual_seed(1337)

# chars = sorted(list(set(text)))

def encode(input):
  return sp.encode(input)
def decode(output):
  return sp.decode(output)

# # Train and test splits
# data = torch.tensor(encode(text), dtype=torch.long)
# n = int(0.9*len(data)) # first 90% will be train, rest val
# train_data = data[:n]
# val_data = data[n:]

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

def save_checkpoint(epoch, model, optimizer, filename='/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/model_checkpoint.pth'):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


In [ ]:
infer_model = BigramLanguageModel()
m = infer_model.to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

6.788896 M parameters


* How to save the model

In [ ]:
# Save the entire model to a .pt file
model_file_path = '/content/drive/MyDrive/StoryTellingMiniGPT/lang_model2.pt'
torch.save(infer_model, model_file_path)

* How to load the weights in the model

In [ ]:
# Load only the 'model_state_dict' from the checkpoint file
checkpoint_path = '/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/storyGPT_wts_16200.pt'
# checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))  # Specify 'map_location' based on your setup

# Load the 'model_state_dict' into your model
m.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))

<All keys matched successfully>

* How to load the model in another file

In [ ]:
#-----#

import torch
# Assuming you already have the model saved in a .pt file (e.g., bigram_model.pt)
model_file_path = '/content/drive/MyDrive/StoryTellingMiniGPT/lang_model2.pt'

# Load the model with 'cpu' as the map_location
loaded_model = torch.load(model_file_path, map_location='cpu')

# Assuming you already have the weights saved in a separate .pt file (e.g., bigram_model_weights.pt)
weights_file_path = '/content/drive/MyDrive/StoryTellingMiniGPT/Wild-folder/storyGPT_wts_16200.pt'

# In another Python file, load the weights with 'cpu' as the map_location
loaded_model_weights = torch.load(weights_file_path, map_location='cpu')

# Set the loaded weights to the loaded model
loaded_model.load_state_dict(loaded_model_weights)

# Now you can use the loaded_model on CPU for predictions


<All keys matched successfully>

* generating from GPT using loaded model

In [ ]:
context = "With new Begining of the life"
vec = torch.tensor(encode(context), dtype=torch.long, device = device)
vec = vec.view(1,-1)
print(decode(loaded_model.generate(vec, max_new_tokens=100)[0].tolist()))

With new Begining of the life's life - The sheer description of dark tales that were forbiddenly matched by the eternal villain. Boethiah beyond MHIWYA, all the who, feared by the thaged within above the count- Rico returning light who dared. ""The already grew stronger from just rock creature you a group of luck that's come away, the warm sun gazed in prisonle's mind — are etched against the hallways or esday, her behely declared unlike the stormed bunny


In [ ]:
ls = [1, 2, 3, 4]
ls

[1, 2, 3, 4]

In [ ]:
ls.append

[1, 2, 3, 4]